<a href="https://colab.research.google.com/github/kostianlomit/habr_bot/blob/master/qwen3__4b_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Introducing FP8 precision training for faster RL inference. [Read Blog](https://docs.unsloth.ai/new/fp8-reinforcement-learning).

Unsloth's [Docker image](https://hub.docker.com/r/unsloth/unsloth) is here! Start training with no setup & environment issues. [Read our Guide](https://docs.unsloth.ai/new/how-to-train-llms-with-unsloth-and-docker).

[gpt-oss RL](https://docs.unsloth.ai/new/gpt-oss-reinforcement-learning) is now supported with the fastest inference & lowest VRAM. Try our [new notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-GRPO.ipynb) which creates kernels!

Introducing [Vision](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl) and [Standby](https://docs.unsloth.ai/basics/memory-efficient-rl) for RL! Train Qwen, Gemma etc. VLMs with GSPO - even faster with less VRAM.

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

### Unsloth


In [2]:
from unsloth import FastLanguageModel
import torch

fourbit_models = [
    "unsloth/Qwen3-4B-Instruct-2507-unsloth-bnb-4bit", # Qwen 14B 2x faster
    "unsloth/Qwen3-4B-Thinking-2507-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    "unsloth/Qwen3-32B-unsloth-bnb-4bit",

    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Instruct-2507",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Добавил адаптеры LoRa, поэтому нам нужно обновить лишь небольшое количество параметров!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.12.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


<a name="Data"></a>
### Data Prep
Теперь мы используем формат Qwen-3 для настройки стиля разговора. Мы используем набор данных FineTome-100k Максима Лабонна в стиле ShareGPT. Qwen-3 отображает многооборотные диалоги, как показано ниже:

```
<|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
Hey there!<|im_end|>

```
Мы используем нашу функцию get_chat_template, чтобы получить правильный шаблон чата. Мы поддерживаем zephyr, chatml, mistral, ламу, альпаку, викунью, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3 и другие.

In [6]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template

# 1. Загрузка вашего датасета
dataset = load_dataset("json", data_files="sample_data/unsloth_gds_dataset.json", split="train")

# 2. Применение шаблона чата к токенизатору
tokenizer = get_chat_template(
    tokenizer,
    chat_template="qwen3-instruct",
)

# 3. Применение шаблона чата к данным датасета
dataset = dataset.map(lambda x: {"text": tokenizer.apply_chat_template(x["conversations"], tokenize=False)})

# 4. Проверка результата
for i in range(3):
    print(f"Пример {i+1}:")
    print(dataset[i]["text"])
    print("-" * 80)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/216 [00:00<?, ? examples/s]

Пример 1:

--------------------------------------------------------------------------------
Пример 2:

--------------------------------------------------------------------------------
Пример 3:

--------------------------------------------------------------------------------


Теперь мы используем standardize_data_formats, чтобы попытаться преобразовать наборы данных в правильный формат для целей точной настройки!

In [7]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/216 [00:00<?, ? examples/s]

Let's see how row 100 looks like!

In [8]:
dataset[100]

{'conversations': [{'content': 'Классифицируй запрос оператора GDS и верни последовательность команд\n\nAlfaTravel::Срочные вопросы - до вылета менее 48 часов - расписание вылета',
   'role': 'user'},
  {'content': 'check_info: КИ/АЭР/ВНУКОВО, КИ/АВК/ДОНАВИА, ТМОВСПТ',
   'role': 'assistant'}],
 'text': ''}

Теперь нам нужно применить шаблон чата для Qwen-3 к разговорам и сохранить его в текстовом формате.

In [10]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/216 [00:00<?, ? examples/s]

Let's see how the chat template did!


In [11]:
dataset[100]['text']

'<|im_start|>user\nКлассифицируй запрос оператора GDS и верни последовательность команд\n\nAlfaTravel::Срочные вопросы - до вылета менее 48 часов - расписание вылета<|im_end|>\n<|im_start|>assistant\ncheck_info: КИ/АЭР/ВНУКОВО, КИ/АВК/ДОНАВИА, ТМОВСПТ<|im_end|>\n'

<a name="Train"></a>
### Теперь давайте потренируем нашу модель. Мы выполняем 60 шагов, чтобы ускорить процесс, но вы можете установить num_train_epochs=1 для полного запуска и отключить max_steps=None.

In [12]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/216 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


Мы также используем метод train_on_completions от Unsloth, чтобы тренировать только выходные данные помощника и игнорировать потери при вводе пользователем. Это помогает повысить точность настройки!

In [13]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=6):   0%|          | 0/216 [00:00<?, ? examples/s]

Давайте проверим, что маскировка части инструкции выполнена! Давайте снова напечатаем 100-ю строку.

In [14]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<|im_start|>user\nКлассифицируй запрос оператора GDS и верни последовательность команд\n\nAlfaTravel::Срочные вопросы - до вылета менее 48 часов - расписание вылета<|im_end|>\n<|im_start|>assistant\ncheck_info: КИ/АЭР/ВНУКОВО, КИ/АВК/ДОНАВИА, ТМОВСПТ<|im_end|>\n'

Теперь давайте распечатаем замаскированный пример - вы должны увидеть, что присутствует только ответ:

In [15]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                       check_info: КИ/АЭР/ВНУКОВО, КИ/АВК/ДОНАВИА, ТМОВСПТ<|im_end|>\n'

In [16]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
3.816 GB of memory reserved.


Давайте потренируем модель! Чтобы возобновить тренировку, установите trainer.train(resume_from_checkpoint = True)

In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 216 | Num Epochs = 3 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


Step,Training Loss
1,5.296400
2,6.409100
3,5.912700
4,5.069400
5,4.207900
6,3.840800
7,3.303600
8,3.083700
9,2.730600
10,2.635600


In [18]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

154.3494 seconds used for training.
2.57 minutes used for training.
Peak reserved memory = 4.445 GB.
Peak reserved memory for training = 0.629 GB.
Peak reserved memory % of max memory = 30.154 %.
Peak reserved memory for training % of max memory = 4.267 %.


<a name="Inference"></a>
### Inference
Давайте запустим модель с помощью встроенного логического вывода Unsloth! Согласно команде Qwen-3, рекомендуемыми настройками для логического вывода команд являются температура = 0,7, top_p = 0,8, top_k = 20

Для логического вывода, основанного на чате, температура = 0,6, top_p = 0,95, top_k = 20

In [27]:
messages = [
    {"role": "user", "content": "Классифицируй запрос оператора GDS и верни последовательность команд\n\nмой агент /статус рейса"}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

inputs = tokenizer(text, return_tensors="pt").to("cuda")

from transformers import TextStreamer
streamer = TextStreamer(tokenizer, skip_prompt=True)

_ = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.1,
    top_p=0.9,
    top_k=50,
    streamer=streamer,
)

check_info: КИ/АЭР/ВНУКОВО, КИ/АВК/ДОНАВИА, ТМОВСПТ<|im_end|>


<a name="Save"></a>
### Saving, loading finetuned models
Чтобы сохранить окончательную модель в виде адаптеров LoRa, используйте push_to_hub от Huggingface для онлайн-сохранения или save_pretrained для локального сохранения.

[ПРИМЕЧАНИЕ] При этом сохраняются только адаптеры LoRa, а не полная модель. Чтобы сохранить файл в формате 16bit или GGUF, прокрутите вниз!

In [20]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Теперь, если вы хотите загрузить адаптеры LoRa, которые мы только что сохранили для вывода, установите значение False равным True:

In [24]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

### Saving to float16 for VLLM

Мы также поддерживаем сохранение непосредственно в float16. Выберите merged_16bit для float16 или merged_4bit для int4. Мы также разрешаем использовать адаптеры lora в качестве запасного варианта. Используйте push_to_hub_merged для загрузки в свой аккаунт Hugging Face! Вы можете перейти по ссылке https://huggingface.co/settings/tokens для получения личных токенов.

In [ ]:
# Merge to 16bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: # Pushing to HF Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False: # Pushing to HF Hub
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
Чтобы сохранить в "GGUF" / "llama.cpp`, мы теперь поддерживаем это изначально! Мы клонируем `llama.cpp" и по умолчанию сохраняем в `q8_0`. Мы разрешаем все методы, такие как "q4_k_m`. Используйте `save_pretrained_gguf` для локального сохранения и `push_to_hub_gguf` для загрузки в HF.

Некоторые поддерживаемые квантовые методы (полный список на нашей [Вики-странице](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Быстрое преобразование. Использование больших ресурсов, но в целом приемлемо.
* `q4_k_m` - Рекомендуется. Половина внимания уделяется Q6_K.wv и feed_forward.тензоры w2, иначе Q4_K.
* Рекомендуется `q5_k_m`. Половину внимания уделяет Q6_K.wv и feed_forward.тензоры w2, остальные Q5_K.

[**НОВЫЙ**] Для точной настройки и автоматического экспорта в Ollama попробуйте наш [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: # Pushing to HF Hub
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Теперь используйте файл model-unsloth.gguf или файл model-unsloth-Q4_K_M.gguf в llama.cpp.

И мы закончили! Если у вас возникнут какие-либо вопросы по Unsloth, у нас есть канал в Discord! Если вы обнаружите какие-либо ошибки или хотите быть в курсе последних разработок LLM, или вам нужна помощь, присоединяйтесь к проектам и т.д., не стесняйтесь присоединяться к нашему дискорду!

Некоторые другие ссылки:

Тренируйте свою собственную модель мышления - Llama GRPO notebook Free Colab
Сохраняйте настройки в Ollama. Бесплатная записная книжка
Llama 3.2 Vision finetuning - пример использования в рентгенографии. Бесплатный Colab
Смотрите в нашей документации записные книжки для DPO, ORPO, непрерывного предварительного обучения, разговорной настройки и многое другое!
